In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
def mean(df_column):
        return(np.sum(df_column)/len(df_column))
                
def median(df_column):
        df_column = df_column.sort_values(ascending=True).reset_index(drop=True)
        if len(df_column) % 2 == 0:
                return ((df_column[len(df_column)//2]+df_column[len(df_column)//2 - 1])/2)
        else: return df_column[len(df_column)//2]

def mode(df_column):
        freq = {}
        for elem in df_column:
                if elem in freq.keys():
                        freq[elem] += 1
                else : freq[elem] = 1
        return max(freq, key=freq.get)

def tendanceCentrale(df_column): #retourne la moyenne, mode et mediane ainsi que la symétrie
        tendances = []
        mean_ = mean(df_column)
        mode_ = mode(df_column)
        median_ = median(df_column)
        tendances.append(mean_)
        tendances.append(mode_)
        tendances.append(median_)
        if round(mean_,1) == round(mode_,1) and round(mean_,1) == round(median_,1) and round(median_,1) == round(mode_,1):
                tendances.append("Symétrique")
        elif round(mean_,1) > round(median_,1) and round(median_,1) > round(mode_,1):
                tendances.append("Positivement")
        elif round(mean_,1) < round(median_,1) and round(median_,1) < round(mode_,1):
                tendances.append("Négativement")
        else: tendances.append("Inindentifiée")
        return pd.Series(np.array(tendances), index=['mean', 'mode', 'median', 'symetrie'])

In [5]:
def ecartType(df_column):
        return(np.sqrt((np.sum(np.power(df_column-mean(df_column),2)))/len(df_column)))

def variance(df_column):
        return(np.power(ecartType(df_column),2))

def getQuartiles(df_column):
        df_column = df_column.sort_values(ascending=True).reset_index(drop=True)
        return (df_column[len(df_column)//4], df_column[(len(df_column)//4)*3])
        
def ecartInterquartile(df_column):
        Q1, Q3 = getQuartiles(df_column)
        return Q3-Q1
        
def dispersion(df_column): #get outlier data and make it into a pandas series
        mesures = []
        standard_deviation = ecartType(df_column)
        variance_ = variance(df_column)
        inter_q = ecartInterquartile(df_column)
        quart = getQuartiles(df_column)
        mesures.append(standard_deviation)
        mesures.append(variance_)
        mesures.append(inter_q)
        mesures.append(min(df_column))
        mesures.append(quart)
        mesures.append(max(df_column))
        outliers = []
        for each in df_column:
                if each > quart[1]+1.5*inter_q or each < quart[0]-1.5*inter_q:
                        outliers.append(each)
        return (pd.Series(np.array(mesures), index=['ecart_type', 'variance', 'IQR', 'min','quartiles', 'max']), set(outliers))

In [6]:

def z_score(dataframe):
        dataframe = pd.DataFrame(dataframe)
        z_scores = []
        means = []
        std_devs = []
        for j in dataframe.columns:
                means.append(mean(dataframe[[j]]).values[0])
                std_devs.append(ecartType(dataframe[[j]]).values[0])
        for i in range(0, dataframe.shape[0]):
                row = []
                c = 0
                for j in dataframe.columns:
                        row.append(((dataframe[[j]].iloc[i]-means[c])/std_devs[c]).values[0])
                        c+=1
                z_scores.append(row)
        normalized = pd.DataFrame(z_score)
        return normalized
        

In [7]:
def correlation(df_column1, df_column2):
        try:
                N = np.int64(len(df_column1))
                xy_sum = np.int64(np.sum(df_column1*df_column2))
                top_sum = np.int64(N*xy_sum)
                sum_x = np.sum(df_column1)
                sum_y = np.sum(df_column2)
                bottom_x = len(df_column1)*np.sum(list(map(lambda x: x*x, df_column1)))
                bottom_y = len(df_column2)*np.sum(list(map(lambda x: x*x, df_column2)))
                return ((top_sum - (sum_x*sum_y))/np.sqrt((bottom_x - np.power(sum_x,2))*(bottom_y - np.power(sum_y,2))))
        except: print("impossible to calculate")

In [145]:
df = pd.read_csv("dataset1.csv")
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [146]:
data_num = df.select_dtypes(exclude=['object'])
data_obj = df.select_dtypes(exclude=['float64', 'int64'])
data_num.columns

Index(['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome',
       'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

In [10]:
data_obj.head()

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
0,Yes,Travel_Rarely,Sales,Life Sciences,Female,Sales Executive,Single,Y,Yes
1,No,Travel_Frequently,Research & Development,Life Sciences,Male,Research Scientist,Married,Y,No
2,Yes,Travel_Rarely,Research & Development,Other,Male,Laboratory Technician,Single,Y,Yes
3,No,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Married,Y,Yes
4,No,Travel_Rarely,Research & Development,Medical,Male,Laboratory Technician,Married,Y,No


In [11]:
mode(data_obj.Attrition)

'No'

### A. Traitement des valeurs manquantes et aberrantes

In [12]:
def replace_missing(df_column, method = 'mode'):
        print(df_column.dtype)
        if df_column.dtype == 'object':
                if method != 'mode': print('this method cannot be used on non-numerical attributes')
                else: 
                        for row in df_column.items():
                                if pd.isnull(row[1]) : 
                                        if method == 'mode':
                                                df_column[row[0]] = mode(df_column)
        else:
                for row in df_column.items():
                        #print(df_column[0])
                        if pd.isnull(row[1]) : 
                                if method == 'mode':
                                        df_column[row[0]] = mode(df_column)
                                elif method == 'median':
                                        df_column[row[0]] = median(df_column)
                                elif method == 'mean':
                                        df_column[row[0]] = mean(df_column)
                                else: print("an error has occured")
        return df_column

def treat_outliers(dataframe, df_column, method = "delete"):
        outliers = dispersion(df_column)[1]
        df = dataframe.copy()
        #iterate through all rows
        for i in range(len(df_column)):
                if df_column.iloc[i] in outliers : 
                        if method == "delete":
                                df=df.drop([i])
                        elif method == "null":
                                df_column.iloc[i] = np.nan
                        elif method == "replace_mode":
                                df_column.iloc[i] = mode(df_column)
                        elif method == "replace_mean":
                                if df_column.dtype == "int64":
                                        df_column.iloc[i] = round(mean(df_column))
                                else: df_column.iloc[i] = mean(df_column)
                        elif method == "replace_median":
                                df_column.iloc[i] = median(df_column)
                                
                        else: print("la methode choisie est invalide")
        df[df_column.name] = df_column

        return df


In [127]:
#replace_missing(df.EnvironmentSatisfaction).isnull().sum()
#treat_outliers(df, df.YearsInCurrentRole,method = "null").isnull().sum()
#replace_missing(df.YearsInCurrentRole).isnull().sum()
df['YearsInCurrentRole']

0       4
1       7
2       0
3       7
4       2
       ..
1465    2
1466    7
1467    2
1468    6
1469    3
Name: YearsInCurrentRole, Length: 1470, dtype: int64

### B. Réduction des données via la discretisation des données continues

In [202]:
# divide the dataset into n quantiles of the same amount of values/rows
# this means each group will have the same amount of values but different interval sizes
# we can reduce a group into a single row by either doing the mean, the median or the mode
def discretisation_effectifs(df_column, Q, method):
        step = (len(df_column)//Q)+1
        column = df_column.sort_values(ascending=True).reset_index(drop=True)
        quantiles = []
        for i in range(0,len(df_column),step):
                quantiles.append(column.iloc[i:i+step])
        for i in range(len(quantiles)): 
                if method == "mean":
                        if df_column.dtype == "int64":
                                quantiles[i] = round(mean(quantiles[i]))
                        else:
                                quantiles[i] = mean(quantiles[i])
                elif method == "mode":
                        quantiles[i] = mode(quantiles[i])
                elif method == "median":
                        quantiles[i] = median(pd.Series(quantiles[i]))
                else: print("la methode choisie est invalide")

        # we return the reduced column and then we just make another function to loop over the df?
        # but in this case we need to ask the chosen method for each column so idk about that lol 
        # they must also have the same number of quantiles too so
        return quantiles

In [133]:
discretisation_effectifs(df.YearsInCurrentRole, 20, 'mean')

[0, 0, 0, 1, 2, 2, 2, 2, 2, 3, 3, 4, 5, 7, 7, 7, 8, 8, 10, 14]

In [200]:
# divide the dataset into n intervals of values
# this means each group will have the same interval sizes but different amount of values
# we can reduce a group into a single row by either doing the mean, the median or the mode
# if k value is not given, use the formula of huntsberger
import math
def discretisation_amplitude(df_column, K, method):
        if K==0:
                K = 1+3*math.log10(len(df_column))
        ## column = df_column.sort_values(ascending=True).reset_index(drop=True)
        interval_size = (max(df_column)- min(df_column))/K
        
        if df_column.dtype == "int64":
                interval_size = math.ceil(interval_size)
        intervals = []
        i = min(df_column)
        while i < max(df_column):
                intervals.append((i, i+interval_size))
                i+=interval_size
        intervals_values = {}
        for each in df_column:
                for interval in intervals:
                        if each >= interval[0] and each < interval[1]:
                                if interval in intervals_values.keys():
                                        intervals_values[interval].append(each)
                                else: 
                                        intervals_values[interval] = [each]
        discretised = []
        for interval in intervals_values.keys():
                if method == "mean":
                        if df_column.dtype == "int64":
                                discretised.append(round(mean(intervals_values[interval])))
                        else: discretised.append(mean(intervals_values[interval]))
                elif method == "median":
                        discretised.append(median(pd.Series(intervals_values[interval])))
                elif method == "mode":
                        discretised.append(mode(intervals_values[interval]))
                else: print("methode choisie invalide")
        discretised.sort()
        return discretised

In [203]:

valeurs_discret=[]
for each in data_num.columns:
    valeurs_discret.append(discretisation_effectifs(df[each],9,'median'))
pd.DataFrame(valeurs_discret,data_num.columns)

,0,1,2,3,4,5,6,7,8
Age,24.0,28.0,31.0,33.5,36.0,38.0,42.0,46.5,54.0
DailyRate,175.0,335.0,504.5,643.0,804.5,961.5,1125.5,1276.5,1419.0
DistanceFromHome,1.0,2.0,2.0,5.0,7.0,9.0,12.0,19.0,26.0
Education,1.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,4.0
EmployeeCount,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EmployeeNumber,105.5,337.5,547.5,796.5,1025.5,1258.5,1501.5,1724.5,1960.5
EnvironmentSatisfaction,1.0,1.0,2.0,3.0,3.0,3.0,4.0,4.0,4.0
HourlyRate,34.0,43.0,51.0,58.0,66.0,74.0,82.0,89.0,97.0
JobInvolvement,1.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0
JobLevel,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,4.0


### C. Réduction des données (redondances) horizontales et verticales

In [18]:
def detect_redundant(df):
        redundances = df.duplicated()
        redundant_columns = []
        for each in df.columns:
                if len(df[each].unique()) == 1 or len(df[each].unique()) == df.shape[0]:
                        redundant_columns.append(each)
        return df[redundances], redundant_columns


In [19]:
df.shape

(1470, 35)

In [137]:
detect_redundant(df)[1]

['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']

In [21]:
def del_row(df, index):
        return df.drop([index], axis = 0)

def del_column(df, index):
        if type(index) is str:
                if index in df.columns:
                        df = df.drop([index], axis = 1)
        elif type(int(index)) is int:
                df = df.drop(df.columns[index], axis = 1)
        return df


In [22]:
#del_column(df, "Age")

### D. Normalisation des données

In [32]:
# follow formula
def min_max_normalisation(df, min_, max_):
        if max_ < min_ : 
                print("min superior to max, please change values")
                return df
        else:
                normalized_df = []
                #df = df.select_dtypes(exclude=['object'])
                min_old = []
                max_old = []
                for column in df.columns:
                        min_old.append(min(df[column]))
                        max_old.append(max(df[column]))
                for i in range(df.shape[0]):
                        row = []
                        c = 0
                        for column in df.columns:
                                if df[column].dtype == 'object':
                                        row.append(df[column].iloc[i])
                                else:
                                        if max_old[c] != min_old[c]:
                                                row.append((((df[column].iloc[i]-min_old[c])/(max_old[c]- min_old[c]))*(max_- min_))+min_)
                                        else:
                                                row.append(min_)
                                c+=1
                        normalized_df.append(row)
                normalized = pd.DataFrame(normalized_df, columns = df.columns)
        return normalized
                                
                        

In [34]:
min_max_normalisation(df, 2,5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,3.642857,Yes,Travel_Rarely,4.147459,Sales,2.000000,2.75,Life Sciences,2,2.000000,...,2.0,2,2.0,2.600,2.0,2.0,2.450,2.857143,2.0,2.882353
1,4.214286,No,Travel_Frequently,2.380100,Research & Development,2.750000,2.00,Life Sciences,2,2.001451,...,5.0,2,3.0,2.750,3.5,4.0,2.750,3.500000,2.2,3.235294
2,3.357143,Yes,Travel_Rarely,4.729420,Research & Development,2.107143,2.75,Other,2,2.004354,...,3.0,2,2.0,2.525,3.5,4.0,2.000,2.000000,2.0,2.000000
3,3.071429,No,Travel_Frequently,4.770222,Research & Development,2.214286,4.25,Life Sciences,2,2.005806,...,4.0,2,2.0,2.600,3.5,4.0,2.600,3.500000,2.6,2.000000
4,2.642857,No,Travel_Rarely,3.050107,Research & Development,2.107143,2.00,Medical,2,2.008708,...,5.0,2,3.0,2.450,3.5,4.0,2.150,2.428571,2.4,2.352941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,3.285714,No,Travel_Frequently,3.679313,Research & Development,4.357143,2.75,Medical,2,4.989840,...,4.0,2,3.0,3.275,3.5,4.0,2.375,2.428571,2.0,2.529412
1466,3.500000,No,Travel_Rarely,3.097351,Research & Development,2.535714,2.00,Medical,2,4.991292,...,2.0,2,3.0,2.675,4.5,4.0,2.525,3.500000,2.2,3.235294
1467,2.642857,No,Travel_Rarely,2.113815,Research & Development,2.321429,3.50,Life Sciences,2,4.994194,...,3.0,2,3.0,2.450,2.0,4.0,2.450,2.428571,2.0,2.529412
1468,4.214286,No,Travel_Frequently,3.977810,Sales,2.107143,3.50,Medical,2,4.995646,...,5.0,2,2.0,3.275,3.5,3.0,2.675,3.285714,2.0,3.411765


In [60]:
#replace with the z score values, already done really but double check if it's the same formula
def z_score_normalisation(dataframe):
        df = pd.DataFrame(dataframe)
        #dataframe = dataframe.select_dtypes(exclude=['object'])
        z_scores = []
        means = []
        std_devs = []
        for j in df.columns:
                if df[j].dtype !='object':
                        means.append(mean(df[[j]]).values[0])
                        std_devs.append(ecartType(df[[j]]).values[0])
        for i in range(0, dataframe.shape[0]):
                row = []
                c = 0
                for j in df.columns:
                        if df[j].dtype == 'object':
                                row.append(df[j].iloc[i])
                        else:
                                row.append(((df[[j]].iloc[i]-means[c])/std_devs[c]).values[0])
                                c+=1
                z_scores.append(row)
        normalized = pd.DataFrame(z_scores, columns = df.columns)
        return normalized
        

In [61]:
z_score_normalisation(df)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.446350,Yes,Travel_Rarely,0.742527,Sales,-1.010909,-0.891688,Life Sciences,NaN,-1.701283,...,-1.584178,NaN,-0.932014,-0.421642,-2.171982,-2.493820,-0.164613,-0.008758,-0.679146,0.245834
1,1.322365,No,Travel_Frequently,-1.297775,Research & Development,-0.147150,-1.868426,Life Sciences,NaN,-1.699621,...,1.191438,NaN,0.241988,-0.164511,0.155707,0.338096,0.488508,0.889418,-0.368715,0.806541
2,0.008343,Yes,Travel_Rarely,1.414363,Research & Development,-0.887515,-0.891688,Other,NaN,-1.696298,...,-0.658973,NaN,-0.932014,-0.550208,0.155707,0.338096,-1.144294,-1.206326,-0.679146,-1.155935
3,-0.429664,No,Travel_Frequently,1.461466,Research & Development,-0.764121,1.061787,Life Sciences,NaN,-1.694636,...,0.266233,NaN,-0.932014,-0.421642,0.155707,0.338096,0.161947,0.889418,0.252146,-1.155935
4,-1.086676,No,Travel_Rarely,-0.524295,Research & Development,-0.887515,-1.868426,Medical,NaN,-1.691313,...,1.191438,NaN,0.241988,-0.678774,0.155707,0.338096,-0.817734,-0.607542,-0.058285,-0.595227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,-0.101159,No,Travel_Frequently,0.202082,Research & Development,1.703764,-0.891688,Medical,NaN,1.721670,...,0.266233,NaN,0.241988,0.735447,0.155707,0.338096,-0.327893,-0.607542,-0.679146,-0.314873
1466,0.227347,No,Travel_Rarely,-0.469754,Research & Development,-0.393938,-1.868426,Medical,NaN,1.723332,...,-1.584178,NaN,0.241988,-0.293077,1.707500,0.338096,-0.001333,0.889418,-0.368715,0.806541
1467,-1.086676,No,Travel_Rarely,-1.605183,Research & Development,-0.640727,0.085049,Life Sciences,NaN,1.726655,...,-0.658973,NaN,0.241988,-0.678774,-2.171982,0.338096,-0.164613,-0.607542,-0.679146,-0.314873
1468,1.322365,No,Travel_Frequently,0.546677,Sales,-0.887515,0.085049,Medical,NaN,1.728317,...,1.191438,NaN,-0.932014,0.735447,0.155707,-1.077862,0.325228,0.590026,-0.679146,1.086895


In [122]:
def z_score_column(df,df_column):
    if(df[df_column].dtype=='object'):
        return df[[df_column]]
    else:
        mean_=mean(df[df_column])
        std=ecartType(df[df_column])
        row=[]
        for i in range (df[df_column].shape[0]):
            row.append(((df[[df_column]].iloc[i]-mean_)/std))
        normalized = pd.DataFrame(row, columns = [df_column])
        print(normalized)
        return normalized
        

In [26]:
from scipy import stats
data = df.select_dtypes(exclude=['object'])
z_scores = pd.DataFrame(stats.zscore(data))
z_scores

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.446350,0.742527,-1.010909,-0.891688,NaN,-1.701283,NaN,1.383138,0.379672,-0.057788,...,-1.584178,NaN,-0.932014,-0.421642,-2.171982,-2.493820,-0.164613,-0.008758,-0.679146,0.245834
1,1.322365,-1.297775,-0.147150,-1.868426,NaN,-1.699621,NaN,-0.240677,-1.026167,-0.057788,...,1.191438,NaN,0.241988,-0.164511,0.155707,0.338096,0.488508,0.889418,-0.368715,0.806541
2,0.008343,1.414363,-0.887515,-0.891688,NaN,-1.696298,NaN,1.284725,-1.026167,-0.961486,...,-0.658973,NaN,-0.932014,-0.550208,0.155707,0.338096,-1.144294,-1.206326,-0.679146,-1.155935
3,-0.429664,1.461466,-0.764121,1.061787,NaN,-1.694636,NaN,-0.486709,0.379672,-0.961486,...,0.266233,NaN,-0.932014,-0.421642,0.155707,0.338096,0.161947,0.889418,0.252146,-1.155935
4,-1.086676,-0.524295,-0.887515,-1.868426,NaN,-1.691313,NaN,-1.274014,0.379672,-0.961486,...,1.191438,NaN,0.241988,-0.678774,0.155707,0.338096,-0.817734,-0.607542,-0.058285,-0.595227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,-0.101159,0.202082,1.703764,-0.891688,NaN,1.721670,NaN,-1.224807,1.785511,-0.057788,...,0.266233,NaN,0.241988,0.735447,0.155707,0.338096,-0.327893,-0.607542,-0.679146,-0.314873
1466,0.227347,-0.469754,-0.393938,-1.868426,NaN,1.723332,NaN,-1.175601,-1.026167,0.845911,...,-1.584178,NaN,0.241988,-0.293077,1.707500,0.338096,-0.001333,0.889418,-0.368715,0.806541
1467,-1.086676,-1.605183,-0.640727,0.085049,NaN,1.726655,NaN,1.038693,1.785511,-0.057788,...,-0.658973,NaN,0.241988,-0.678774,-2.171982,0.338096,-0.164613,-0.607542,-0.679146,-0.314873
1468,1.322365,0.546677,-0.887515,0.085049,NaN,1.728317,NaN,-0.142264,-1.026167,-0.057788,...,1.191438,NaN,-0.932014,0.735447,0.155707,-1.077862,0.325228,0.590026,-0.679146,1.086895


In [129]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
names = data.columns
d = scaler.fit_transform(data)
scaled_df = pd.DataFrame(d, columns=names)
scaled_df.head(6)


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.547619,0.715820,0.000000,0.25,0.0,0.000000,0.333333,0.914286,0.666667,0.25,...,0.000000,0.0,0.000000,0.200,0.000000,0.000000,0.150,0.285714,0.000000,0.294118
1,0.738095,0.126700,0.250000,0.00,0.0,0.000484,0.666667,0.442857,0.333333,0.25,...,1.000000,0.0,0.333333,0.250,0.500000,0.666667,0.250,0.500000,0.066667,0.411765
2,0.452381,0.909807,0.035714,0.25,0.0,0.001451,1.000000,0.885714,0.333333,0.00,...,0.333333,0.0,0.000000,0.175,0.500000,0.666667,0.000,0.000000,0.000000,0.000000
3,0.357143,0.923407,0.071429,0.75,0.0,0.001935,1.000000,0.371429,0.666667,0.00,...,0.666667,0.0,0.000000,0.200,0.500000,0.666667,0.200,0.500000,0.200000,0.000000
4,0.214286,0.350036,0.035714,0.00,0.0,0.002903,0.000000,0.142857,0.666667,0.00,...,1.000000,0.0,0.333333,0.150,0.500000,0.666667,0.050,0.142857,0.133333,0.117647
5,0.333333,0.646385,0.035714,0.25,0.0,0.003387,1.000000,0.700000,0.666667,0.00,...,0.666667,0.0,0.000000,0.200,0.333333,0.333333,0.175,0.500000,0.200000,0.352941
